# sparkで機械学習がしたい
http://mogile.web.fc2.com/spark/ml-classification-regression.html

In [179]:
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_iris
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import confusion_matrix, classification_report

from IPython.display import display

In [180]:
data_iris_all = load_iris(as_frame=True)

data_iris = data_iris_all["data"]
target_iris = data_iris_all["target"]
target_label_iris = data_iris_all["target_names"]
frame_iris = data_iris_all["frame"]

print(data_iris_all.keys())
display(data_iris.head())
print(data_iris.shape)
print(target_iris.value_counts())

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


(150, 4)
0    50
1    50
2    50
Name: target, dtype: int64


In [181]:
display(frame_iris.head())
display(frame_iris.info())

#from pyspark.sql.types import *
#from pyspark.ml import Pipeline
#from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer, IDF
#from pyspark.ml.classification import NaiveBayes
#rom pyspark.ml.evaluation import BinaryClassificationEvaluator

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-null    float64
 3   petal width (cm)   150 non-null    float64
 4   target             150 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 6.0 KB


None

### Sparkのセッションを起動
javaが必要

In [182]:
from pyspark.sql import SparkSession
# セッションを開く
try:
    #spark.stop()
    pass
except NameError:
    pass

spark = SparkSession.builder.appName("pyspark-notebook").master(master="local[*]").getOrCreate()
spark

In [183]:
# print runtime versions
print ('****************')
print ('Python version: {}'.format(sys.version))
print ('Spark version: {}'.format(spark.version))
print ('****************')


****************
Python version: 3.8.5 (default, Jul 21 2020, 10:48:26) 
[Clang 11.0.3 (clang-1103.0.32.62)]
Spark version: 3.1.1
****************


In [184]:
import pyspark.sql.functions as F
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler, VectorIndexer

In [185]:
#データの読み込み（pandasのDataFrameより）
spark_data = spark.createDataFrame(frame_iris)

columns = spark_data.columns

In [186]:
print("columns : ", spark_data.columns)
print("rows : ", spark_data.count())
spark_data.show(10)

columns :  ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)', 'target']
rows :  150
+-----------------+----------------+-----------------+----------------+------+
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|target|
+-----------------+----------------+-----------------+----------------+------+
|              5.1|             3.5|              1.4|             0.2|     0|
|              4.9|             3.0|              1.4|             0.2|     0|
|              4.7|             3.2|              1.3|             0.2|     0|
|              4.6|             3.1|              1.5|             0.2|     0|
|              5.0|             3.6|              1.4|             0.2|     0|
|              5.4|             3.9|              1.7|             0.4|     0|
|              4.6|             3.4|              1.4|             0.3|     0|
|              5.0|             3.4|              1.5|             0.2|     0|
|              

In [187]:
"""
特徴量をまとめる
"""
feature_cols = columns[:-1]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
spark_data_2 = assembler.transform(spark_data)

print("columns : ", spark_data_2.columns)
print("rows : ", spark_data_2.count())
spark_data_2.show(10)

columns :  ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)', 'target', 'features']
rows :  150
+-----------------+----------------+-----------------+----------------+------+-----------------+
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|target|         features|
+-----------------+----------------+-----------------+----------------+------+-----------------+
|              5.1|             3.5|              1.4|             0.2|     0|[5.1,3.5,1.4,0.2]|
|              4.9|             3.0|              1.4|             0.2|     0|[4.9,3.0,1.4,0.2]|
|              4.7|             3.2|              1.3|             0.2|     0|[4.7,3.2,1.3,0.2]|
|              4.6|             3.1|              1.5|             0.2|     0|[4.6,3.1,1.5,0.2]|
|              5.0|             3.6|              1.4|             0.2|     0|[5.0,3.6,1.4,0.2]|
|              5.4|             3.9|              1.7|             0.4|     0|[5.4,3.9,1.7,0.4]|

In [188]:
"""
学習用データの作成
pysparkのlabelColのデフォルトが'label'なので変更しておく
"""
learning_data = spark_data_2.select("features", F.col("target").alias("label"))
learning_data.show(10)

train, test = learning_data.randomSplit([0.8, 0.2], seed=10)

+-----------------+-----+
|         features|label|
+-----------------+-----+
|[5.1,3.5,1.4,0.2]|    0|
|[4.9,3.0,1.4,0.2]|    0|
|[4.7,3.2,1.3,0.2]|    0|
|[4.6,3.1,1.5,0.2]|    0|
|[5.0,3.6,1.4,0.2]|    0|
|[5.4,3.9,1.7,0.4]|    0|
|[4.6,3.4,1.4,0.3]|    0|
|[5.0,3.4,1.5,0.2]|    0|
|[4.4,2.9,1.4,0.2]|    0|
|[4.9,3.1,1.5,0.1]|    0|
+-----------------+-----+
only showing top 10 rows



In [189]:
from pyspark.ml.classification import GBTClassifier, RandomForestClassifier
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

#### ランダムフォレスト

In [205]:
rf = RandomForestClassifier(
    featuresCol="features"
    , labelCol="label"
    , numTrees=10
)

model = rf.fit(train)

In [206]:
predictions = model.transform(test)
#予測結果の確認
predictions.toPandas().sample(10)

,features,label,rawPrediction,probability,prediction
11,"[5.1, 3.8, 1.9, 0.4]",0,"[10.0, 0.0, 0.0]","[1.0, 0.0, 0.0]",0.0
26,"[5.7, 2.5, 5.0, 2.0]",2,"[0.0, 0.23896103896103896, 9.761038961038961]","[0.0, 0.023896103896103894, 0.9761038961038961]",2.0
2,"[4.7, 3.2, 1.6, 0.2]",0,"[10.0, 0.0, 0.0]","[1.0, 0.0, 0.0]",0.0
20,"[5.5, 2.6, 4.4, 1.2]",1,"[0.0, 8.975, 1.025]","[0.0, 0.8975, 0.1025]",1.0
6,"[5.5, 3.5, 1.3, 0.2]",0,"[10.0, 0.0, 0.0]","[1.0, 0.0, 0.0]",0.0
10,"[5.1, 3.8, 1.6, 0.2]",0,"[10.0, 0.0, 0.0]","[1.0, 0.0, 0.0]",0.0
28,"[6.3, 2.7, 4.9, 1.8]",2,"[0.0, 2.0851148851148853, 7.914885114885115]","[0.0, 0.20851148851148854, 0.7914885114885115]",2.0
25,"[7.1, 3.0, 5.9, 2.1]",2,"[0.0, 0.05714285714285714, 9.942857142857143]","[0.0, 0.005714285714285714, 0.9942857142857143]",2.0
5,"[5.0, 3.4, 1.5, 0.2]",0,"[10.0, 0.0, 0.0]","[1.0, 0.0, 0.0]",0.0
14,"[5.6, 2.5, 3.9, 1.1]",1,"[0.0, 9.975, 0.025]","[0.0, 0.9974999999999999, 0.0025]",1.0


In [207]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="label"
    , predictionCol="prediction"
    , metricName="accuracy"
)
accuracy = evaluator.evaluate(predictions)
print(accuracy)

1.0


#### GBDT回帰

In [199]:
#とりあえずデフォルトパラメータで学習
gbtr = GBTRegressor(
    featuresCol="features"
    , labelCol="label"
    , maxIter=100
)

#学習
model = gbtr.fit(train)

In [200]:
predictions = model.transform(test)
predictions.toPandas().sample(10)

,features,label,prediction
5,"[5.0, 3.4, 1.5, 0.2]",0,0.0
1,"[4.6, 3.4, 1.4, 0.3]",0,0.0
34,"[7.9, 3.8, 6.4, 2.0]",2,2.0
16,"[6.1, 2.9, 4.7, 1.4]",1,1.0
15,"[5.8, 2.7, 4.1, 1.0]",1,1.0
20,"[5.5, 2.6, 4.4, 1.2]",1,1.0
4,"[4.9, 3.1, 1.5, 0.2]",0,0.0
30,"[6.7, 3.3, 5.7, 2.1]",2,2.0
0,"[4.4, 2.9, 1.4, 0.2]",0,0.0
24,"[6.5, 3.2, 5.1, 2.0]",2,2.0


In [201]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="label"
    , predictionCol="prediction"
    , metricName="accuracy"
)
accuracy = evaluator.evaluate(predictions)
print(accuracy)

0.9428571428571428


In [175]:
spark.stop()